# Table of Contents

- [Interpolation](#Interpolation)

- [Linear Interpolation (InterpLinear)](#Linear-Interpolation-(InterpLinear))
  - [set_points_and_triangles() Method](#set_points_and_triangles()-Method)
  - [set_use_natural_neighbor() Method](#set_use_natural_neighbor()-Method)
  - [scalars Property](#scalars-Property)
  - [interpolate_to_point() Method](#interpolate_to_point()-Method)
  - [interpolate_to_points()Method](#interpolate_to_points()-Method)
  - [set_truncation(), truncate_min, truncate_max, and truncate_interpolated_values](#set_truncation(),-truncate_min,-truncate_max,-and-truncate_interpolated_values)
  - [point_activity Property](#point_activity-Property)
  - [triangle_activity Property](#triangle_activity-Property)
  - [extrapolation_point_indexes Property](#extrapolation_point_indexes-Property)
  - [triangle_containing_point() Method](#triangle_containing_point()-Method)
  - [triangle_envelopes_containing_point() Method](#triangle_envelopes_containing_point()-Method)
  - [interpolate_weights() Method](#interpolate_weights()-Method)
  - [extrapolation_value Property](#extrapolation_value-Property)
  - [set_use_clough_tocher() Method](#set_use_clough_tocher()-Method)
  
- [Inverse Distance Weighted Interpolation (InterpIdw)](#Inverse-Distance-Weighted-Interpolation-(InterpIdw))
  - [Truncation](#Truncation)
  - [set_points() Method](#set_points()-Method)
  - [points Property](#points-Property)
  - [scalars Property](#idw_scalars)
  - [interpolate_to_point() Method](#idw_interpolate_to_point)
  - [interpolate_to_points() Method](#idw_interpolate_to_points)
  - [set_nodal_function() Method](#set_nodal_function()-Method)
  - [weight_calculation_method Property](#weight_calculation_method-Property)
  - [power Property](#power-Property)
  - [set_search_options() Method](#set_search_options()-Method)
  - [search_options_use_quadrant_search Property](#search_options_use_quadrant_search-Property)
  - [nodal_function_number_nearest_points Property](#nodal_function_number_nearest_points-Property)
  - [point_activity Property](#idw_point_activity)
  - [interpolate_weights_method Property](#idw_interpolate_weights_method)
  
- [InterpLinearExtrapIdw](#InterpLinearExtrapIdw)
  - [InterpLinearExtrapIdw Methods and Properties](#InterpLinearExtrapIdw-Methods-and-Properties)
  - [set_idw_nodal_function() Method](#set_idw_nodal_function()-Method)
  - [set_idw_search_options() Method](#set_idw_search_options()-Method)
  
- [InterpAnisotropic](#InterpAnisotropic)
  - [InterpAnisotropic.set_points()](#InterpAnisotropic.set_points())
  - [set_x_scale() Method](#set_x_scale()-Method)
  - [set_power() Method](#set_power()-Method)
  - [interp_to_point() Method](#interp_to_point()-Method)
  - [interp_to_points() Method](#interp_to_points()-Method)
  - [get_interpolation_points() Method](#get_interpolation_points()-Method)
  - [get_transformed_points() Method](#get_transformed_points()-Method)
  
- [Convenience Methods](#Convenience-Methods)
  - [interpolate_to_points() Method](#convenience_interpolate_to_points)

- [Example - 2D Grid Interpolation](#Example---2D-Grid-Interpolation)
  - [Generate grid for visualization](#Generate-grid-for-visualization)

# Interpolation

When using xmsinterp to interpolate a scatter dataset there are a few steps in general to follow. These steps are shown below.

- Get the scatter point data that we will interpolate.
- Get the points/grid that we will interpolate to.
- Setup any additional parameters.
- Interpolate the data.

The different methods of interpolation that can be done with xmsinterp are demonstrated in the following sections. The final section, [Example - 2D Grid Interpolation](#Example---2D-Grid-Interpolation), shows some examples of interpolation with some more realistic data. For more info on interpolation see the <a href="https://www.xmswiki.com/wiki/GMS:Interpolation">xmswiki</a>.

# Linear Interpolation (InterpLinear)

The <a href="https://www.xmswiki.com/wiki/GMS:Linear">Linear interpolation</a> scheme uses data points that are first triangulated to form a network of triangles. The network of triangles only covers the convex hull of the point data, making extrapolation beyond the convex hull not possible. To perform linear interpolation we use a `InterpLinear` object. The constructor of the `InterpLinear` object takes the following arguments:

- `points` (list): The points of the source geometry as x,y,z tuples.
- `triangles` (list): The triangles of the source geometry as point index tuples.
- `scalars` (list): The scalar values at the points.
- `nodal_function` (str): The nodal function to use One of: 'constant', 'gradient_plane', 'quadratic'.
- `point_search_option` (str): The point search option. One of: 'natural_neighbor', 'nearest_points'.
- `number_nearest_points` (int): The number of nearest points to consider.
- `blend_weights` (bool): True if weights should be blended.
- `progress` (Observer): Observer object for providing feedback.

The only required argument is `points`, with all the other arguments having default values. Creation of an InterpLinear object is shown below.

In [1]:
# Import needed for InterpLinear
from xms.interp.interpolate.interp_linear import InterpLinear

# Create an InterpLinear object
interper = InterpLinear(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)])

Using InterpLinear's properties, we can see that even though we didn't pass values for the constructor's other arguments, values for triangles and scalars were automatically determined.

In [2]:
print(f'triangles: {interper.triangles}')
print(f'scalars: {interper.scalars}')

triangles: [3 0 1 1 2 3]
scalars: [0. 1. 2. 3.]


Next we'll need to create the points/bounding grid that our scatter dataset will interpolate to. These will be x,y,z tuples stored in an iterable.

In [3]:
points = ((2, 1, 0), (5, 10, 2.5))

Finally, we will interpolate our data using the `interpolate_to_points()` method.

In [4]:
ret = interper.interpolate_to_points(points)
ret

(0.5, 2.5)

The return value will be a tuple holding a value for each point that was in the iterable passed to `interpolate_to_points()`.

#### set_points_and_triangles() Method

The points and triangles can alternatively be set with the `set_points_and_triangles()` method. The following arguments are accepted:

- `points` (iterable): Array of point locations.
- `triangles` (iterable): Array of triangles that references the points array.

The array passed as the triangle argument must have a size that is a multiple of 3. The first 3 locations in the array represent the first triangle and will have indices that correspond to locations in the points array.  The usage of this method is shown below:

In [5]:
interper.set_points_and_triangles(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)], triangles=[3, 0, 1, 1, 2, 3])

ret = interper.interpolate_to_points(points)
ret

(0.5, 2.5)

#### set_use_natural_neighbor() Method

If we want to interpolate using <a href="https://www.xmswiki.com/wiki/GMS:Natural_Neighbor">Natural Neighbor</a> we can use the `set_use_natural_neighbor()` method. This method takes the following arguments:

- `on` (bool): Indicate if Natural Neighbor should be used.
- `nodal_function` (str): Indicates which nodal function to use. One of 'constant', 'gradient_plane', or 'quadratic'.
- `point_search_option` (str): Indicates options for the nearest points when computing the nodal functions. One of 'natural_neighbor', or 'nearest_points'.
- `number_nearest_points` (int): The number of nearest points for nodal function computation.
- `blend_weights` (bool): Option to use a blending function on the calculated weights.
- `progress` (observer): Progress bar to give user feedback for generation of the nodal functions.

All of these arguments are optional and will be given default values when no values are passed to them.

In [6]:
interper.set_use_natural_neighbor(on=True, nodal_function='constant')

ret = interper.interpolate_to_points(points)
ret

(0.1559019237756729, 2.5)

#### scalars Property

The `scalars` property can be used to set the values that will be interpolated from. It takes an iterable whose length should be equal to the number of points passed to either the constructor of `InterpLinear`, or the number of points passed to `set_points_and_triangles()` method. The usage of this property is shown below.

In [7]:
interper.scalars = [1.0, 3.4, 2.0, 4.0]

#### interpolate_to_point() Method

The `interpolate_to_point()` method uses the stored triangles to interpolate to the given point. It takes tuple of three values for the point to interpolate to, and returns the interpolated value at the given point. If the given point is outside the stored triangles (extrapolation) then the <a href="#extrapolation_value-Property">`extrapolation_value`</a> is returned.

In [8]:
# interpolate to a point inside the stored triangles
print(f'{interper.interpolate_to_point((1.0, 1.0, 0.0))}')

# interpolate to a point outside the stored triangles (extrapolation). extrapolation_value is returned
interper.extrapolation_value = -9999
print(f'{interper.interpolate_to_point((100, 100, 100))}')

1.1299608945846558
-9999.0


#### interpolate_to_points() Method

The `interpolate_to_points()` method is the same as <a href="#interpolate_to_point()-Method">`interpolate_to_point()`</a>, except it can be passed multiple points.

In [9]:
interper.interpolate_to_points([(1.0, 1.0, 1.0), (2.0, 2.0, 2.0), (100, 100, 100)])

(1.1299608945846558, 1.4432363510131836, -9999.0)

#### set_truncation(), truncate_min, truncate_max, and truncate_interpolated_values

At times it can be desirable to limit the results to a certain range. This can be done with the `set_truncation(maximum,minimum)` method.

In [10]:
interper.set_truncation(minimum=0, maximum=3)

ret = interper.interpolate_to_points(points)
ret

(1.2856812477111816, 3.0)

You can see what truncation values are set with the `truncate_min` and `truncate_max` properties. The `truncate_interpolated_values` property will tell you if the results are being truncated.

In [11]:
print(f'truncate_min: {interper.truncate_min}')
print(f'truncate_max: {interper.truncate_max}')
interper.truncate_interpolated_values

truncate_min: 0.0
truncate_max: 3.0


True

#### point_activity Property

The `point_activity` property can be used to get/set the activity of the points being interpolated from. It takes an iterable whose length is the number of points being interpolated from. The values represent the activity of the point with the corresponding index.

In [12]:
interper.point_activity = (True, True, False, True)

interper.point_activity

array([1, 1, 0, 1], dtype=uint8)

#### triangle_activity Property

The `triangle_activity` property can be used to get/set the activity of the triangles being interpolated from. It takes an iterable whose length is the number of triangles being interpolated from. The values represent the activity of the triangle with the corresponding index.

In [13]:
interper.triangle_activity = (True, True)

interper.triangle_activity

array([1, 1], dtype=uint8)

#### extrapolation_point_indexes Property

The `extrapolation_point_indexes` property returns a list of indexes for the points that were outside all of the stored triangles. The points that are considered are the points that were passed to the `interpolate_to_points()` method. Usage of this property is illustrated below.

In [14]:
# The points at index 0 and 3 are outside the stored triangles
interper.interpolate_to_points([(500, 500, 500), (1.0, 1.0, 1.0), (2.0, 2.0, 2.0), (100, 100, 100)])

interper.extrapolation_point_indexes

array([0, 3])

#### triangle_containing_point() Method

The `triangle_containing_point()` method returns the index of the stored triangle that the given point is located in. If none of the triangles contain the point then `-1` is returned.

In [15]:
# Passing a point that does lie inside a stored triangle
print(interper.triangle_containing_point((1.0,1.5,1.0)))

# Passing a point that DOES NOT lie inside a stored triangle
print(interper.triangle_containing_point((100,100,100)))

0
-1


#### triangle_envelopes_containing_point() Method

The `triangle_envelopes_containing_point()` Method returns a list of the indexes of the stored triangles that the given point is located in. If none of the triangles contain the point then an empty list is returned.

In [16]:
# Passing a point that does lie inside a stored triangle
print(interper.triangle_envelopes_containing_point((1.0,1.5,1.0)))

# Passing a point that DOES NOT lie inside a stored triangle
print(interper.triangle_envelopes_containing_point((100,100,100)))

[0 3]
[]


#### interpolate_weights() Method

The `interpolate_weights()` method takes a point and uses the stored triangles to get the interpolation weights for the given point. A tuple containing three entries is returned. The first entry is a bool indicating whether or not the point was inside the stored triangles. The second entry is a list containing the indexes of the stored points whose weights were interpolated. The third entry contains a list of the weights for the point's whose indexes were in the list of the second entry.  In the case that the first entry is `False`, both the lists in the second and third entry will be empty.

In [17]:
# Passing a point that lies inside the stored triangles
r1 = interper.interpolate_weights((1,1,1))

# Passing a point that DOES NOT lie inside the stored triangles
r2 = interper.interpolate_weights((100,100,100))

print(r1)
print(r2)

(True, array([3, 0, 1]), array([0.1, 0.8, 0.1]))
(False, array([], dtype=int32), array([], dtype=float64))


#### extrapolation_value Property

The `extrapolation_value` property can be used to set what value is returned when the object attempts to interpolate outside the stored triangles. <a href="#Linear-Interpolation-(InterpLinear)">Linear interpolation</a> cannot extrapolate beyond the convex hull formed by the stored triangles, instead the `extrapolation_value` is returned in the case extrapolation is attempted.

In [18]:
interper.extrapolation_value = -9999

#### set_use_clough_tocher() Method

The `set_use_clough_tocher()` method allows you to tell the `InterpLinear` object to use the <a href="https://www.xmswiki.com/wiki/GMS:Clough-Tocher">Clough Tocher</a> method of interpolation. It takes two arguments which are shown below:

- `on` (bool): Whether or not clough tocher interpolation should be used.
- `progress` (observer): Progress bar to give users feed back on the set up process of CT. If you have a really large set of triangles this may take some time.

This is a legacy feature from GMS. Its usage is shown below.

In [19]:
interper.set_use_clough_tocher(on=True)

ret = interper.interpolate_to_point((1.0, 1.0, 1.0))
ret

0.16200000047683716

# Inverse Distance Weighted Interpolation (InterpIdw)

<a href="https://www.xmswiki.com/wiki/GMS:Inverse_Distance_Weighted">Inverse Distance Weighted (IDW)</a> is one of the most commonly used techniques for interpolation of point data. Its methods are based on the assumption that the interpolating surface should be influenced most by the nearby points and less by the more distant points. To perform inverse distance weighted interpolation, we use a `InterpIdw` object. The `InterpIdw` constructor takes the following arguments.

- `points` (list): The points of the source geometry as x,y,z tuples.
- `scalars` (list): The scalar values at the points.
- `nodal_function` (str): The nodal function to use One of: 'constant', 'gradient_plane', 'quadratic'.
- `number_nearest_points` (int): The number of nearest points to consider.
- `quadrant_oct` (bool): True if quadrant search should be used.
- `progress` (Observer): Observer object for providing feedback.
- `is_2d` (bool): flag for 2d vs 3d interpolation.
- `weight_number_nearest_points` (int): number of points to include in interpolation weight calculation.
- `weight_quadrant_oct` (bool): get nearest points from quadrants for weight calculation.

All arguments are optional except for points. The code to create a simple `InterpIdw` object is shown below.

In [20]:
# Import needed for InterpIdw
from xms.interp.interpolate.interp_idw import InterpIdw

# Create an InterpIdw object
interper = InterpIdw(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)], scalars=[1, 2, 3, 4])

Next we'll need to create the points/bounding grid that our scatter dataset will interpolate to. These will be x,y,z tuples stored in an iterable.

In [21]:
points = ((2, 1, 0), (5, 10, 2.5))

Now we can pass `points` to `interpolate_to_points()` to run IDW interpolation.

In [22]:
ret = interper.interpolate_to_points(points)
ret

(1.0268155336380005, 3.5)

For more information on `interpolate_to_points()` see its section below.

#### Truncation

At times it can be desirable to limit the results to a certain range. This can be done with the `set_truncation(maximum,minimum)` method.

In [23]:
interper.set_truncation(minimum=0, maximum=3)

ret = interper.interpolate_to_points(points)
ret

(1.0268155336380005, 3.0)

You can see what truncation values are set with the `truncate_min`, `truncate_max`, and `truncate_interpolation_values` properties.

In [24]:
print(f'truncate_min: {interper.truncate_min}')
print(f'truncate_max: {interper.truncate_max}')
print(f'truncate_interpolation_values: {interper.truncate_interpolation_values}')

truncate_min: 0.0
truncate_max: 3.0
truncate_interpolation_values: True


#### set_points() Method

The `set_points(points, is_2d)` method can be used to set the points being interpolated from outside the constructor. It takes the following two arguments:

- `points` (iterable): Array of point locations.
- `is_2d` (bool): Flag if this is 2D.

Usage of this method is illustrated below.

In [25]:
interper.set_points([(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)], False)

ret = interper.interpolate_to_points(points)
ret

(1.0224628448486328, 3.0)

#### points Property

The `points` property can be used to retrieve the points that are being interpolated from.

In [26]:
interper.points

array([[ 0.,  0.,  0.],
       [10.,  0.,  1.],
       [10., 10.,  2.],
       [ 0., 10.,  3.]])

<a id="idw_scalars"></a>
#### scalars Property

The `scalars` property can be used to set the values that will be interpolated from. It takes an iterable whose length should be equal to the number of points passed to either the constructor of `InterpIdw`, or the number of points passed to the `set_points()` method. The usage of this property is shown below.

In [27]:
interper.scalars = [1.0, 3.4, 2.0, 4.0]

<a id="idw_interpolate_to_point"></a>
#### interpolate_to_point() Method

The `interpolate_to_point(point)` method interpolates to the given point. It takes tuple of three values for the point to interpolate to, and returns the interpolated value at the given point.

In [28]:
interper.interpolate_to_point((1.0, 1.0, 0.0))

1.011813759803772

<a id="idw_interpolate_to_points"></a>
#### interpolate_to_points() Method

The `interpolate_to_points()` method is the same as [`interpolate_to_point()`](#idw_interpolate_to_point), except it can be passed multiple points.

In [29]:
interper.interpolate_to_points([(1.0, 1.0, 1.0), (2.0, 2.0, 2.0), (100, 100, 100)])

(1.0202679634094238, 1.1311293840408325, 2.617445468902588)

#### set_nodal_function() Method

IDW interpolation has many options that can be customized through the properties and methods of the `InterpIdw` class. The type of nodal function to be used can be set with the `set_nodal_function()` method. A nodal function is a plane or quadratic function that is forced to pass through the scatter point and approximate the nearby scatter points in a least squares sense. `set_nodal_function()` accepts the following arguments:

- `nodal_function` (string): The nodal function methodology: 'constant' (0), 'gradient_plane' (1), 'quadratic' (2).
- `number_nearest_points` (int): The nearest number of points to use when calculating the nodal functions.
- `quadrant_oct` (bool): Find the nearest number of points in each quadrant (2d) or octant (3d) when computing nodal functions.
- `progress` (Observer): Progress bar to give user feedback.

In [30]:
interper.set_nodal_function(nodal_function='quadratic', number_nearest_points=16, quadrant_oct=False)

ret = interper.interpolate_to_points(points)
ret

(1.0399633646011353, 3.0)

#### weight_calculation_method Property

The weight calculation method can be set with the `weight_calculation_method` property. The value of this property determines how the weights for each point are calculated. The valid values for this property are either `'classic'` or `'modified'`. The classic option uses 1/distance^exponent. The modified method uses another formulation based on the distance of the furtherest location from the interpolation point.

In [31]:
interper.weight_calculation_method = 'classic'

ret = interper.interpolate_to_points(points)
ret

(1.3225356340408325, 2.872264862060547)

#### power Property

By default the class does inverse distance squared weighting, but the exponent can be changed to any value with the `power` property.

In [32]:
interper.power = 4

ret = interper.interpolate_to_points(points)
ret

(1.0231636762619019, 2.9719362258911133)

#### set_search_options() Method

You can use the `set_search_options()` method to set how to find the nearest points to the interpolation point. `set_search_options()` accepts the following arguments:

- `nearest_point` (int): The number of nearest points to the interpolation point. These points are used to do the interpolation.
- `quadrant_oct_search` (bool): Specifies if the search criterion should find the nearest points in each quadrant (2d) or octant (3d)

Usage of this method is shown below:

In [33]:
interper.set_search_options(16, True)

ret = interper.interpolate_to_points(points)
ret

(1.0231636762619019, 2.9719362258911133)

#### search_options_use_quadrant_search Property

The `search_options_use_quadrant_search` property will return `True` if the search criterion is set to find the nearest points in each quadrant (2d). It will return `False` if the search criterion is set to find the nearest points in each octant (3d).

In [34]:
interper.set_search_options(nearest_point=16, quadrant_oct_search=False)
print(interper.search_options_use_quadrant_search)

interper.set_search_options(nearest_point=16, quadrant_oct_search=True)
print(interper.search_options_use_quadrant_search)

False
True


#### nodal_function_number_nearest_points Property

The `nodal_function_number_nearest_points` property returns the nodal function number nearest the points.

In [35]:
interper.nodal_function_number_nearest_points

16

<a id="idw_point_activity"></a>
#### point_activity Property

The `point_activity` property can be used to get/set the activity of the points being interpolated from. It takes an iterable whose length is the number of points being interpolated from. The values represent the activity of the point with the corresponding index.

In [36]:
interper.point_activity = (True, True, False, True)

interper.point_activity

<bound method PyCapsule.GetPtActivity of <xms.interp._xmsinterp.interpolate.InterpIdw object at 0x00000279C260D7F0>>

<a id="idw_interpolate_weights_method"></a>
#### interpolate_weights() Method

The `interpolate_weights()` method takes a point and gets the interpolation weights for the given point. A tuple containing two entries is returned. The first entry is a list containing the indexes of the stored points whose weights were interpolated. The second entry contains a list of the weights for the point's whose indexes were in the list of the second entry.

In [37]:
interper.interpolate_weights((1,1,1))

(array([0, 1, 3]), array([0.99745115, 0.00133508, 0.00121377]))

# InterpLinearExtrapIdw

The `InterpLinearExtrapIdw` class is a combination of both the `InterpLinear` class and the `InterpIdw` class. It uses Linear Interpolation when interpolating, and Inverse Distance Weighted interpolation when extrapolating. The code below illustrates this with a simple example. 

In [38]:
# Import needed for InterpLinearExtrapIdw
from xms.interp.interpolate.interp_linear_extrap_idw import InterpLinearExtrapIdw

# Interpolating and extrapolating with InterpLinear. When extrapolating, the extrapolation_value is returned.
print('InterpLinear')
interperLinear = InterpLinear(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)])
print(interperLinear.interpolate_to_point((1, 1, 1)))
print(interperLinear.interpolate_to_point((100, 100, 100)))

# Interpolating and extrapolating with InterpIdw. When extrapolating, an actual prediction is returned.
print('\nInterpIdw')
interperIdw = InterpIdw(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)])
print(interperIdw.interpolate_to_point((1, 1, 1)))
print(interperIdw.interpolate_to_point((100, 100, 100)))

# Interpolating and extrapolating with InterpLinearExtrapIdw. When extrapolating, an actual prediction is returned.
print('\nInterpLinearExtrapIdw')
interper = InterpLinearExtrapIdw(points=[(0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3)])
print(interper.interpolate_to_point((1, 1, 1)))
print(interper.interpolate_to_point((100, 100, 100)))

InterpLinear
0.4000000059604645
nan

InterpIdw
0.010227557271718979
2.0

InterpLinearExtrapIdw
0.4000000059604645
2.0


#### InterpLinearExtrapIdw Methods and Properties

`InterpLinearExtrapIdw` shares many methods and properties with both `InterpLinear` and `InterpIdw`. For information on how to use these methods and properties, see the <a href="#Linear-Interpolation-(InterpLinear)">Linear Interpolation</a> section and the <a href="#Inverse-Distance-Weighted-Interpolation-(InterpIdw)">Inverse Distance Weighted Interpolation</a> section. The sections below touch on a few methods that have been renamed in the `InterpLinearExtrapIdw` class.

#### set_idw_nodal_function() Method

The `set_idw_nodal_function()` method allows you to set the type of nodal function as well as the options for computing nodal functions. These options are for when inverse distance weighting interpolation is used. It is the same method as <a href="#set_nodal_function()-Method">`set_nodal_function()`</a> in the <a href="#Inverse-Distance-Weighted-Interpolation-(InterpIdw)">`InterpIdw` class</a>. See <a href="#set_nodal_function()-Method">`set_nodal_function()`</a> for more info on nodal functions.  The `set_idw_nodal_function()` method takes four arguments which are listed below.

- `nodal_function` (string): The nodal function methodology: 'constant' (0), 'gradient_plane' (1), 'quadratic' (2).
- `number_nearest_points` (int): The nearest number of points to use when calculating the nodal functions.
- `quadrant_oct` (bool): Find the nearest number of points in each quadrant (2d) or octant (3d) when computing nodal functions.
- `progress` (Observer): Progress bar to give user feedback.

An example of its usage is given below.

In [39]:
interper.set_idw_nodal_function(nodal_function='quadratic', number_nearest_points=16, quadrant_oct=False)

ret = interper.interpolate_to_points(points=[(2, 1, 0), (5, 10, 2.5)])
ret

(0.5, 2.5)

#### set_idw_search_options() Method

The `set_idw_search_options()` method allows you to set the search options for how to find the nearest points to the interpolation point. These options are for when inverse distance weighting interpolation is used. It is the same method as <a href="#set_search_options()-Method">`set_search_options()`</a> in the <a href="#Inverse-Distance-Weighted-Interpolation-(InterpIdw)">`InterpIdw` class.</a> It takes the two arguments listed below.

- `nearest_point` (int): The number of nearest points to the interpolation point. These points are used to do the interpolation.
- `quadrant_oct_search` (bool): Specifies if the search criterion should find the nearest points in each quadrant (2d) or octant (3d).

A simple example on how to use this method is shown below.

In [40]:
interper.set_idw_search_options(16, True)

ret = interper.interpolate_to_points(points)
ret

(0.5, 2.5)

# InterpAnisotropic

Sometimes the data associated with a scatter point set will have directional tendencies. Anisotropic interpolation allows  us to take these tendencies into account. Anisotropic interpolation is actually inverse distance weighting interpolation with some extra steps. To perform anisotropic interpolation we use an `InterpAnisotropic` object. The `InterpAnisotropic` constructor does not take any arguments and can be called like so:

In [41]:
# Import needed for InterpAnisotropic
from xms.interp.interpolate.interp_anisotropic import InterpAnisotropic

# Create an InterpAnisotropic object
interper = InterpAnisotropic()

<a id="InterpAnisotropic.set_points()"></a>
#### set_points() Method

The `set_points()` method is used to set both the points to be interpolated from, and the points that make up the center line. It takes three arguments which are listed below:

- `center_line_points` (iterable): Points that make the centerline.
- `interpolation_points` (iterable): Points that will be used to interpolate from.
- `pick_closest` (bool): If true, only keep one transformed interpolation point (that is closest to the centerline).

Below is an example that uses `set_points()`.

In [42]:
# Create the centerline points.  The segments of this polyline will be mapped
# onto the x-axis.  The total length will be preserved.

centerline = [[0, 0, 0], [20, 0, 0], [40, 20, 0], [60, 20, 0]]

# Create the points used to interpolate. These points are typically taken
# from cross sections across the center line.  These points will be
# transformed so that the x-coordinate corresponds to the intersection of a
# line through the point and perpendicular to a segment of the centerline.
# The y coordinate will be the distance above or below that intersection.
# Note that a single point may project onto several segments, generating
# multiple transformed points.

interpolation_points = [
    # cross-section 1
    [0, 10, 100],
    [0, -10, 100],
    # cross-section 2
    [10, 10, 90],
    [10, 5, 60],
    [10, 0, 50],
    [10, -5, 70],
    [10, -10, 90],
    # cross-section 3
    [20, 20, 80],
    [30, 10, 40],
    [40, 0, 80],
    # cross-section 4
    [30, 40, 70],
    [35, 30, 50],
    [40, 20, 30],
    [45, 10, 70],
    # cross-section 5
    [60, 30, 50],
    [60, 10, 50]]

# Set the centerline and interpolation points used by the interpolator.
# Any point may project onto several segments of the centerline. You can use
# all such transformed points or set pick_closest to true to use only the
# one nearest the centerline.

interper.set_points(centerline, interpolation_points, pick_closest=False)

#### set_x_scale() Method

The `set_x_scale()` method allows you to scale the points in the x direction. The default x scale factor is 1. Setting the scale to a value less than 1 will compress the x values, thus giving them more weight than y. Setting the scale to a value greater than 1 will result in the opposite effect.

In [43]:
# Set the x scale to 0.5, thus compressing the x values.
interper.set_x_scale(0.5)

#### set_power() Method

The `set_power()` method allows you to set the exponent of the inverse distance weight. It takes one argument which is the exponent used to compute the point weights. This method fills the same role as the <a href="#power-Property">`set_power`</a> property in the <a href="#Inverse-Distance-Weighted-Interpolation-(InterpIdw)">`InterpIdw`</a> class.

In [44]:
# Set the exponent of the inverse distance weight to a value over 1 to dilute
# the influence of interpolation points far from the point in question.
interper.set_power(3)

#### interp_to_point() Method

The `interp_to_point(point)` method interpolates to the given point. It takes tuple of three values for the point to interpolate to, and returns the interpolated value at the given point.

In [45]:
interper.interp_to_point((20, 5, 0))

59.53133773803711

#### interp_to_points() Method

The `interp_to_points()` method is the same as `interp_to_point()`, except it can be passed multiple points.

In [46]:
# Interpolate to several locations.  Note that the last
# two points are beyond the range of the centerline, hence
# they generate no data (XM_NODATA) interpolation values.

# The points we are interpolating to
points = [
    [5, 5, 0], [5, 0, 0], [5, -5, 0],
    [20, 5, 0],
    [20, 0, 0],
    [20, -5, 0], [10, 20, 0], [30, -5, 0], [30, 30, 0],
    [35, 20, 0], [45, 25, 0], [45, 15, 0],
    [65, 20, 0], [-5, 0, 0]
]

# Interpolate to the points
interper.interp_to_points(points)

(64.61399841308594,
 54.390106201171875,
 71.97010803222656,
 59.53133773803711,
 58.46803283691406,
 68.09170532226562,
 81.6883544921875,
 76.03239440917969,
 53.13092803955078,
 35.41265106201172,
 40.45703125,
 50.83898162841797,
 -9999999.0,
 -9999999.0)

#### get_interpolation_points() Method

The `get_interpolation_points()` method transforms and scales the points passed to the [`set_points()`](#InterpAnisotropic.set_points()) method into (s,n,z) space. The resulting points are then returned in a list. This method is not required for interpolation, but can be used for when you want to use the transformed/scaled data.

In [47]:
interper.get_interpolation_points()

array([[  0.        ,  10.        , 100.        ],
       [  0.        , -10.        , 100.        ],
       [  5.        ,  10.        ,  90.        ],
       [ 10.        ,  14.14213562,  90.        ],
       [  5.        ,   5.        ,  60.        ],
       [  5.        ,   0.        ,  50.        ],
       [  5.        ,  -5.        ,  70.        ],
       [  5.        , -10.        ,  90.        ],
       [ 10.        ,  20.        ,  80.        ],
       [ 17.07106781,  14.14213562,  80.        ],
       [ 17.07106781,   0.        ,  40.        ],
       [ 17.07106781, -14.14213562,  80.        ],
       [ 24.14213562, -20.        ,  80.        ],
       [ 24.14213562,  22.36067977,  70.        ],
       [ 24.14213562,  11.18033989,  50.        ],
       [ 24.14213562,   0.        ,  30.        ],
       [ 24.14213562,   0.        ,  30.        ],
       [ 22.37436867, -10.60660172,  70.        ],
       [ 26.64213562, -10.        ,  70.        ],
       [ 34.14213562,  10.     

#### get_transformed_points() Method

The `get_transformed_points()` method transforms and scales a list of points. This method accepts the following two arguments:

- `points` (iterable): The points to transform into (s,n,z) space.
- `get_closest` (bool): True to pick only the closest transformed point.

The resulting points are returned in a list. The `get_transformed_points()` method allows us to use interpolation methods other then [inverse distance weighting](#Inverse-Distance-Weighted-Interpolation-(InterpIdw)) with our horizontally stretched data.

In [48]:
# Create the points that we will pass to get_transformed_points()
points = [
    [5, 5, 0], [5, 0, 0], [5, -5, 0],
    [20, 5, 0],
    [20, 0, 0],
    [20, -5, 0], [10, 20, 0], [30, -5, 0], [30, 30, 0],
    [35, 20, 0], [45, 25, 0], [45, 15, 0],
    [65, 20, 0], [-5, 0, 0]
]

interper.get_transformed_points(points, False)

array([[  2.5       ,   5.        ,   0.        ],
       [  2.5       ,   0.        ,   0.        ],
       [  2.5       ,  -5.        ,   0.        ],
       [ 10.        ,   5.        ,   0.        ],
       [ 11.76776695,   3.53553391,   0.        ],
       [ 10.        ,   0.        ,   0.        ],
       [ 10.        ,   0.        ,   0.        ],
       [ 10.        ,  -5.        ,   0.        ],
       [  5.        ,  20.        ,   0.        ],
       [ 13.53553391,  21.21320344,   0.        ],
       [ 11.76776695, -10.60660172,   0.        ],
       [ 24.14213562,  14.14213562,   0.        ],
       [ 22.37436867,   3.53553391,   0.        ],
       [ 26.64213562,   5.        ,   0.        ],
       [ 24.14213562,  -7.07106781,   0.        ],
       [ 26.64213562,  -5.        ,   0.        ]])

# Convenience Methods

There are a few methods available in xmsinterp that are designed to make it easier to use interpolation. The following section goes over how to use one of the convenience methods available in xmsinterp.

<a id="convenience_interpolate_to_points"></a>
#### interpolate_to_points() Method

The `interpolate_to_points()` method allows you to interpolate values from one set of points to another. The benefit of using this method is the ease of interpolating with different interpolation types from one function call. This benefit is shown in the following example, where `interpolate_to_points()` is used to interpolate with linear interpolation, inverse distance interpolation, and natural neighbor interpolation. `interpolate_to_points()` takes three required arguments, and three optional arguments. These arguments are listed below:

- `interpolation_points` (Iterable): The points to be used for interpolation
- `points` (Iterable): The points to interpolate to
- `method` (string): The interpolation method to be used. One of: `'linear'`, `'idw'`, or `'natural_neighbor'`.
- `x` (string): The x dataset (optional). Ex: `'x'`
- `y` (string): The y dataset (optional). Ex: `'y'`
- `z` (string): The z dataset (optional). Ex: `'z'`

In [49]:
# Import needed for interpolate_to_points()
from xms.interp.api.interpolator import interpolate_to_points

# Create the points being interpolated from
points_from = ((0, 0, 0), (10, 0, 1), (10, 10, 2), (0, 10, 3))

# Create the points being interpolated to
points_to = ((2, 1, 0), (5, 10, 2.5), (5, 5), (7, 3), (2, 7), (7, 7))

# Call interpolate_to_points (linear)
print(f"linear: {interpolate_to_points(method='linear', interpolation_points=points_from, points=points_to)}")

# Call interpolate_to_points (idw)
print(f"idw: {interpolate_to_points(method='idw', interpolation_points=points_from, points=points_to)}")

# Call interpolate_to_points (natural neighbor)
print(f"natural neighbor: {interpolate_to_points(method='natural_neighbor', interpolation_points=points_from, points=points_to)}")

linear: (0.5, 2.5, 2.0, 1.600000023841858, 2.299999952316284, 2.0)
idw: (0.02681550197303295, 2.5, 1.5, 1.0, 2.829150915145874, 2.0)
natural neighbor: (0.5, 2.5, 2.0, 1.600000023841858, 2.299999952316284, 2.0)


# Example - 2D Grid Interpolation

The xmsinterp library contains classes for performing 2d (x,y) linear, natural neighbor, and idw interpolation: InterpLinear and InterpIdw. Natural neighbor interpolation is performed using the linear interpolation class. 3d (x,y,z) interpolation can be performed using the idw class. Points (x,y) must be given to create the interpolation class.

We will learn how to use the interpolation classes by using a set of 2D points (x,y) with a measurement at each point. These points come from a csv file. Then we will generate a grid that bounds these points and use different interpolation options to visualize the data.

In [50]:
import numpy as np
from xms import interp

The following code reads in data which we will interpolate from a csv file.

In [51]:
xs = []
ys = []
cs = []

# Read in the data for our scatter plot
with open("plumedat.csv") as input_file:
    # Ignore column headers
    input_file.readline()
    
    for line in input_file:
        line = line.rstrip()
        id,x,y,c = line.split(",")
        xs.append(int(x))
        ys.append(int(y))
        cs.append(float(c))

# Get input values as list of (x,y,z)
values = list(zip(xs,ys,cs))
print(values)

[(-75, -16, 0.0), (-60, 32, 0.0), (-34, 50, 0.0), (-34, 27, 0.0), (-8, 40, 0.0), (16, 38, 0.0), (-25, 14, 43.64), (10, 18, 44.16), (27, 26, 0.0), (63, 35, 0.0), (-32, 0, 59.04), (-7, 7, 90.2), (26, 6, 67.2), (75, 7, 0.0), (-37, -15, 9.24), (-7, -13, 71.0), (2, -3, 98.4), (31, -15, 25.56), (60, -13, 0.0), (-50, -30, 0.0), (-30, -28, 0.0), (43, -22, 0.0), (-32, -50, 0.0), (27, -37, 0.0), (60, -33, 0.0)]


Below is an image that shows what our scatter plot data looks like when plotted with holoviews

![Scatter Plot Data](images/scatter_plot_1.png)

## Generate grid for visualization
Get the min, max (x, y) of the data and generate a grid of points.

In [52]:
# get the (x,y) range of the data and expand by 10%
def buffer_range(range_min, range_max, buf_by=0.1):
    buf = (range_max - range_min) * buf_by
    return (range_min - buf, range_max + buf)

# Get the min/max coordinates
x_min = min(xs)
x_max = max(xs)
y_min = min(ys)
y_max = max(ys)

x_range = buffer_range(x_min, x_max)
y_range = buffer_range(y_min, y_max)

# adjust these values for a higher/lower resolution grid
number_of_x_points=600
number_of_y_points=400
x = np.linspace(x_range[0], x_range[1], number_of_x_points)
y = np.linspace(y_range[0], y_range[1], number_of_y_points)
grid = np.meshgrid(x, y)
pts = np.stack([i.flatten() for i in grid], axis=1)

Interpolate to the grid using linear interpolation and display the results with matplotlib.

In [53]:
linear = interp.interpolate.InterpLinear(values)
z = linear.interpolate_to_points(pts)
# I would prefer to be able to set up an interpolator like this:
# linear = interp.Interpolator(kind='linear', values=df.values)

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Linear Interpolation](images/interpolation_1.png)

Interpolate using natural neighbor. The first example shows a "constant" nodal function and the second example shows a "quadratic" nodals function. A nodal function can better model trends that exist in the measured values.

In [54]:
linear.set_use_natural_neighbor(on=True)
z = linear.interpolate_to_points(pts)
# z = linear.interpolate_to_points(pts, use='natural_neighbor')

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Linear Interpolation Using Natural Neighbor](images/interpolation_2.png)

Now do natural neighbor with a quadratic nodal function.

In [55]:
linear.set_use_natural_neighbor(on=True, nodal_function="quadratic", 
                                point_search_option="nearest_points")
z = linear.interpolate_to_points(pts)
# z = linear.interp_to_pts(pts, use='natural_neighbor', nodal_func_type='quadratic', nd_func_pt_search_opt='nearest_pts')

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Linear Interpolation using natural neighbor and a quadratic nodal function](images/interpolation_3.png)

Now we will use Inverse Distance Weighted (IDW) to interpolate to the grid. Notice that unlike Linear and Natural Neighbor interpolation, IDW interpolation can extrapolate beyond the bounds of the input data points.

In [56]:
idw = interp.interpolate.InterpIdw(values)
z = idw.interpolate_to_points(pts)
# idw = interp.Interpolator(kind='idw', values=df.values)
# z = idw.interpolate_to_points(pts)

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Inverse Distance Weighted Interpolation](images/interpolation_4.png)

Now interpolate using a nodal function to try to better capture trends in the data set.

In [57]:
idw.set_nodal_function(nodal_function="gradient_plane")
z = idw.interpolate_to_points(pts)
# z = idw.interp_to_pts(pts, nodal_func_type='gradient_plane')

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Inverse Distance Weighted Interpolation with a Nodal Function](images/interpolation_5.png)

Notice, in the previous interpolation that we get negative values in some locations. If we assume that there should be no negatives values (like when measuring concentration) we can use the truncation options to adjust the interpolation.

In [58]:
idw.set_truncation(maximum=150, minimum=0)
z = idw.interpolate_to_points(pts)
# z = idw.interpolate_to_points(pts, smax=150, smin=0)

# The commented out code below displays the interpolation with matplotlib resulting in the image below.
#zz = np.reshape(z, (len(y), len(x)))
#xa = xr.DataArray(coords={'x': x, 'y': y}, dims=['x', 'y'], data=zz.T)
#xa.hvplot(x='x', y='y', height=500, cmap='viridis')

![Inverse Distance Weighted Interpolation with a Nodal Function and Truncation](images/interpolation_6.png)